# goal of this notebook:
### in this notebook for every file we want to find all the minimizers of each kmer and use that to write the kmer and the appropriate taxid of this kmer, in certain place of file.
suppous for each minimizer we create folder and we store all the kmer associated to this minimizer in that folder, now for writing new kmer , for example for kmer number 1 in 204669_kmer.fq we find minimizer minimizer(k1) and check the directory of ./minimizer(k1)/ . if the folder not exist , we  make directory minimizer(k1) and put kmer1 and  204669 in this directory. but if the minimizer(k1) exist already, we check if kmer1 exist already in this directory? if yes , we change the taxid of kmer 1 in this directory to LCA(ExistTaxid,204669). if kmer not exist in minimizer(k1) directory we add kmer 1 and 204669 to the directory.

### this was abstract review of what we will do in this document

## importing libraries:

In [1]:
from ete3 import NCBITaxa
import random
import glob
import time
import mmh3
import sys
import numpy as np
import h5py
import os

this function give us common ancestor of each two node in tree. If two taxid is equal we return that node and if it was diffrent we use built in function from NCBItaxa package to get taxid of common ancestor of those two nodes

In [2]:
def getLCA(taxid1,taxid2,tree):
    if taxid1 == taxid2:
        return taxid1
    else:
        return int(tree.get_common_ancestor(str(taxid1),str(taxid2)).name)

### the problem of using minimizer as key for kmers in lexical order is that :
#### it tend to find low complexity String as minimizer for example AAAAAAA. and we have many low complexity strings in all the genomes. so it result to very large directory /AAAAAAA/ . the reason of this event is  there is lot of kmers in genomes that has AAAAAAA substring. if the size of /AAAAAAA/ become very large it is dificult to load all this data in directory to cache of cpu.
for prevent this problem we produce random number and by xoring this random number to each string the lexical order of string will be shuffled. for example in some case AAAAAA is biger than TAAAAA
in this cell we produce that random number

In [3]:
random.seed(1)
n = 31
const = int(''.join(["{}".format(random.randint(0, 9)) for num in range(0, n)]))

and this cell given string we replace all character with number and we xor const by the reason I explained above

In [4]:
def getNum(st):
    conv = {"A":1,"C":2,"G":3,"T":4}
    temp = 0
    l = len(st)
    for i in range(len(st)):
        temp+=conv[st[i]]*(10**(l-i-1))
    return int((temp ^ const%(10**(len(st)))))

### the reason that I dont use jellyfish is that jelly fish change the order of kmers.
### but why we need the kmers be ordered?
### we can find minimizers in two ways:
 1- slide windows across forward and reverse complement of string and find minimum substring among all of those<br>
 2- hold last string and its minimizer that we calculate minimizer for that and given new string, we only check the last and start of oldstring and new string(with just two window).  if minimizer was in first of old string with high probablity the minimizer of new string is changed and we use method 1 but if it is not we then check las of new string and compare its minimizer with old minimizer . if new minimizer is smaller than old minimizer we update minimizer

the method 2 is not depend on size of string and is much faster for larg n but to use approach 2 the kmers should be ordered.I implemnt these two aproach in next two cells. (we use method 1 when we dont have old string)

In [5]:
def getMinimizer(seq):
    rev=seq[::-1]
    rev=rev.replace("A","X")
    rev=rev.replace("T","A")
    rev=rev.replace("X","T")
    rev=rev.replace("C","X")
    rev=rev.replace("G","C")
    rev=rev.replace("X","G")
    Kmer=len(seq)
    M= (Kmer/5) +1
    L=len(seq)
    min=int(''.join(["{}".format(random.randint(9, 9)) for num in range(0, M)]))
    for j in range(0, Kmer-M+1):
        if j == 0:
            sub2=seq[j:j+M]
            if getNum(sub2) < min:
                min=sub2
            sub2=rev[j:j+M]
            if getNum(sub2) < min:
                min=sub2
        else:
            sub2=seq[j:j+M]
            if getNum(sub2) < getNum(min):
                min=sub2
            sub2=rev[j:j+M]
            if getNum(sub2) < getNum(min):
                min=sub2
    return min

In [6]:
def updateMinimizer(seq,oldseq,minimizer):
    revseq=seq[::-1]
    revseq=revseq.replace("A","X")
    revseq=revseq.replace("T","A")
    revseq=revseq.replace("X","T")
    revseq=revseq.replace("C","X")
    revseq=revseq.replace("G","C")
    revseq=revseq.replace("X","G")
    Kmer=len(seq)
    M= (Kmer/5) +1
    firstMinimizerOfOld = oldseq[0:M]
    revseqOld=firstMinimizerOfOld[::-1]
    revseqOld=revseqOld.replace("A","X")
    revseqOld=revseqOld.replace("T","A")
    revseqOld=revseqOld.replace("X","T")
    revseqOld=revseqOld.replace("C","X")
    revseqOld=revseqOld.replace("G","C")
    revseqOld=revseqOld.replace("X","G")
    L=len(seq)
    end_f = seq[L-M:L]
    start_f = seq[0:M]
    end_r = revseq[L-M:L]
    start_r = revseq[0:M]
    minNum = getNum(minimizer)    
    if (getNum(revseqOld) == minNum) or (getNum(firstMinimizerOfOld) == minNum):
        if(getMinimizer(seq) == minNum):
            return False,minimizer
        else:
            return True,getMinimizer(seq)
    flag = False
    if (getNum(end_f) < minNum):
        minimizer = end_f
    if(getNum(start_r) < minNum):
        flag = True
        minimizer = start_r
    return flag,minimizer

by making small each kmer and each minimizer our performance become better and better.
instead of writing each kmer and minimizer in file I first hash them and then use their hash as key and value in file. on average this work take half of size than last approach

In [7]:
def getHashedKmer(kmer):
    return mmh3.hash(kmer,seed=1,signed=True)

In [8]:
def getHashedMinimizer(minimizer):
    return mmh3.hash(minimizer,seed=2,signed=True)

### the main part:
#### now we have all the functions that we need
as I explain first of this note book we should consider one path for each minimizer and store all the kmer,taixd into that path. I use a dictionary that has key and value. the key of this dictionary is my minimizer and the value of this dictionary is another dictionary that contain all the kmers related to that minimizer as key and taxids as value.
#### the goal of this part is to construct this huge dictionary in Ram
first i Itterate on the kmer files and for each kmer I find Hashed minimizer and Hashed kmer and I will check if the index[minimizer] not exist I make it and sore the {Hkmer:taxid} on it.
if it exist I go to the directory and check if index[Hmin][Hkmer] exist or not? if it exist I calculate Lca of new taxid and old taxid and if it not exist i just write it to that directory

In [9]:
fileNames = (glob.glob("../genomes/*.fq"))
ncbi = NCBITaxa()
ParentNode="Acidobacteria" #inja ro bayad karbar bde az kojaye derakht b paeen mikhad
ParentTaxid = ncbi.get_name_translator([ParentNode])[ParentNode][0]
tree = ncbi.get_descendant_taxa(ParentNode, return_tree=True)
index = {}
for PathFilename in fileNames:
    taxid = int(PathFilename.split("_")[0].split("/")[2])
    start_time = time.time()
    with open(PathFilename) as f:
        count = 0
        oldkmer = ""
        oldMinimizer = ""
        for kmer in f:
            kmer = kmer.strip()
            if count == 0:
                minimizer = getMinimizer(kmer)
                oldkmer = kmer
                oldMinimizer = minimizer
            else:
                flag,minimizer = updateMinimizer(kmer,oldkmer,oldMinimizer)
                oldkmer = kmer
                oldMinimizer = minimizer
            Hmin = getHashedMinimizer(minimizer)
            Hkmer =getHashedKmer(kmer)
            if Hmin in index:
                if Hkmer in index[Hmin]:
                    index[Hmin][Hkmer] = getLCA(index[Hmin][Hkmer],taxid,tree)
                else:
                    index[Hmin][Hkmer] = taxid
            else:
                index[Hmin] = {Hkmer:taxid}
            count +=1
    print "file  " + str(taxid)+ "_kmer.fq" + "processed in this time:"
    print("--- %s seconds ---" % (time.time() - start_time))

file  1198114_kmer.fqprocessed in this time:
--- 250.392916203 seconds ---
file  1855912_kmer.fqprocessed in this time:
--- 337.673550129 seconds ---
file  1592106_kmer.fqprocessed in this time:
--- 290.066696167 seconds ---
file  981222_kmer.fqprocessed in this time:
--- 164.681408882 seconds ---
file  401053_kmer.fqprocessed in this time:
--- 232.843212128 seconds ---
file  204669_kmer.fqprocessed in this time:
--- 255.945392847 seconds ---
file  234267_kmer.fqprocessed in this time:
--- 448.328229189 seconds ---
file  240015_kmer.fqprocessed in this time:
--- 188.423882961 seconds ---
file  926566_kmer.fqprocessed in this time:
--- 241.118476152 seconds ---
file  682795_kmer.fqprocessed in this time:
--- 287.801056147 seconds ---
file  2211140_kmer.fqprocessed in this time:
--- 349.414181948 seconds ---


### we can not hold this dictionary in RAM beacause we should represent this file beside our package and user just use this file to classify read of samples.
 ### but there is a problem if user read all the index file, because it cause all the index file come into Ram again.
### we need file structure that has this ability to load certain part of file I used HDF5 file format.
 in this file format we can have two things: 1- group 2- data set. data sets is just like numpy array in python and group is just path in linux or windows. for example I stored numpy array in group "aaa". I can access that array by this adress: "/aaa/". the benefit of using this file format is that we dont need to Load all the the index file. in the next notebook we want to find taxids related to certain read. so we find each kmer of read and find minimizer and then load /minimizer/ data set in index file not all the file. for next kmer with high probablity there is no need to load another data set beacause it may hava same minimizer as last kmer.
in this cell we convert all the inner dictionary to numpy array with two row. row 1 refers to Hashed kmer and row 2 refers to taxids.

In [10]:
start_time = time.time()
with h5py.File("../resources/index.hdf5", "w") as f:    
    for key in index:
        minim = key
        arr = np.row_stack((index[key].keys(),index[key].values()))
        dset = f.create_dataset(str(minim)+'/list',(2,arr.shape[1]), dtype='i')
        dset[...] = arr
print "dictionary in index file finished in time:"
print("--- %s seconds ---" % (time.time() - start_time))
index = []

dictionary in index file finished in time:
--- 52.171806097 seconds ---


## the result of this note book is index file located in resources path of project
### in the next notebook named readClassifier we use Fastq file that contain noisy reads from these 11 files with specific abundance of each genomes and will write code to predict the abundance from Fastq file and index. then we compare our result to the real abundancy of each genomes
please open readClassifier notebook